### **pip installs**

In [ ]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes


!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update -qq
!apt install openjdk-8-jdk-headless -qq


     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 43.9 MB/s 
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 155225 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u312-b07-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u312-b07-0ubuntu1~18.04) ...
Selecting previously unselected package openjdk-8-jdk-headless:amd64.

**code imports**

In [ ]:
from collections import Counter, OrderedDict
from google.cloud import storage
from google.colab import auth
from graphframes import *
from inverted_index_colab import *
from itertools import islice, count, groupby
from nltk.corpus import stopwords
from nltk.stem.porter import *
from operator import itemgetter
from pathlib import Path
from pyspark import SparkContext, SparkConf
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from time import time
from timeit import timeit
import builtins
import hashlib
import itertools
import math
import nltk
import numpy as np
import os
import pandas as pd
import pickle
import pyspark
import re
import sys

In [ ]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar


--2022-01-08 08:00:28--  https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 13.226.124.26, 13.226.124.24, 13.226.124.67, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|13.226.124.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 247880 (242K) [binary/octet-stream]
Saving to: ‘/usr/local/lib/python3.7/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar’

graphframes-0.8.2-s 100%[===================>] 242.07K  --.-KB/s    in 0.04s   

2022-01-08 08:00:28 (5.98 MB/s) - ‘/usr/local/lib/python3.7/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar’ saved [247880/247880]



In [ ]:
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

# Authenticate your user
# The authentication should be done with the email connected to your GCP account
auth.authenticate_user()

# Copy one wikidumps files 


project_id = 'core-period-321814'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass  
except:
      !mkdir wikidumps
      !gsutil cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey

Copying gs://wikidata_preprocessed/multistream1_preprocessed.parquet...
- [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


In [ ]:
from pathlib import Path 
try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)

doc_title_pairs = parquetFile.limit(1000).select("title", "id").rdd
doc_text_pairs = parquetFile.limit(1000).select("text", "id").rdd
doc_anchor_text_before = parquetFile.limit(1000).select("id","anchor_text").rdd
doc_anchor_text_pairs = (doc_anchor_text_before.map(lambda x: (map(lambda y: y[1],x[1]),x[0]))).map(lambda x: (" ".join(x[0]),x[1]))


In [ ]:
#parquetFile.show()

In [ ]:
#print(doc_title_pairs.take(5))

**Code from assignment 3**

---



In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)


def doc_count(text, id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE
  countTokens = OrderedDict(Counter(tokens))
  dict_keys = list(countTokens.keys())
  for token in dict_keys:
    if token in all_stopwords:
      countTokens.pop(token)
  countTokens_len = builtins.sum(countTokens.values())
  return (id,countTokens_len)

def word_count(text, id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  countTokens = OrderedDict(Counter(tokens))
  dict_keys = list(countTokens.keys())
  for token in dict_keys:
    if token in all_stopwords:
      countTokens.pop(token)
  return list(map(lambda x: (x , (id,countTokens[x])),countTokens.keys()))


def reduce_word_counts(unsorted_pl):
  return sorted(unsorted_pl, key=lambda x: x[1], reverse=True)

def calculate_df(postings):
  return postings.map(lambda x: (x[0],len(x[1])))


NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings,basedir):
  newpostings = postings.map(lambda x: (token2bucket_id(x[0]),x))
  words2buckets = newpostings.groupByKey()
  x=words2buckets.map(lambda x: InvertedIndex.write_a_posting_list(x,basedir))
  return x

def countForDict(posting_locs_list):
  super_posting_locs = defaultdict(list)
  for posting_loc in posting_locs_list:
    for k, v in posting_loc.items():
      super_posting_locs[k].extend(v)
  return super_posting_locs

def createIndex(doc_rdd,directory):
  inverted = InvertedIndex()
  word_counts = doc_rdd.flatMap(lambda x: word_count(x[0], x[1]))
  postings = word_counts.groupByKey().mapValues(reduce_word_counts)
  #postings_filtered = postings.filter(lambda x: len(x[1])>10)
  df_ = calculate_df(postings)
  inverted.df = df_.collectAsMap()
  word_counter = postings.map(lambda x: (x[0], builtins.sum([y[1] for y in x[1]])))
  inverted.term_total = Counter(word_counter.collectAsMap())
  posting_locs_list = partition_postings_and_write(postings,directory).collect()
  inverted.posting_locs = countForDict(posting_locs_list)
  return inverted

def writeIdx(inverted, directory, name):
  inverted.write_index(directory, name) 
  inverted.write_dct(directory, name+'_dct') 


In [ ]:
from inverted_index_colab import *
!mkdir text title anchor

In [ ]:
# #write to directories
title_index = createIndex(doc_title_pairs,'title')
title_index.dct=doc_title_pairs.map(lambda x: (x[1],x[0])).collectAsMap()
writeIdx(title_index, 'title','index_title')


text_index = createIndex(doc_text_pairs,'text')
word_counts_length = doc_text_pairs.map(lambda x: doc_count(x[0], x[1])) #only body
text_index.dct = word_counts_length.collectAsMap()  #only body
writeIdx(text_index, 'text','index_text')


anchor_index = createIndex(doc_anchor_text_pairs,'anchor')
writeIdx(anchor_index, 'anchor','index_anchor')

In [ ]:
TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

In [ ]:
# t = InvertedIndex.read_posting_list(text_index,'movie')
# print(t)

**CosSim Code:**

In [ ]:

def tokenize(text):
    list_of_tokens =  [token.group() for token in RE_WORD.finditer(text.lower()) if token.group() not in all_stopwords]    
    return list_of_tokens

def get_posting_gen(index):
    words, pls = zip(*index.posting_lists_iter())
    return words, pls


def generate_query_tfidf_vector(query_to_search, index, DL):
    epsilon = .0000001
    total_vocab_size = len(index.term_total)
    Q = np.zeros(total_vocab_size)
    term_vector = list(index.term_total.keys())
    counter = Counter(query_to_search)
    for token in np.unique(query_to_search):
        if token in index.term_total.keys():  # avoid terms that do not appear in the index.
            tf = counter[token] / len(query_to_search)  # term frequency divded by the length of the query
            df = index.df.get(token, 0)
            idf = math.log((len(DL)) / (df + epsilon), 10)  # smoothing

            try:
                ind = term_vector.index(token)
                Q[ind] = tf * idf
            except:
                pass
    return Q


def get_candidate_documents_and_scores(query_to_search, index, words, pls, DL):
    candidates = {}
    N = len(DL)
    for term in np.unique(query_to_search):
        if term in words:
            list_of_doc = pls[words.index(term)]
            normlized_tfidf = []
            for doc_id, freq in list_of_doc:
                normlized_tfidf.append((doc_id, (freq / DL[doc_id]) * math.log(N / index.df[term], 10)))

            for doc_id, tfidf in normlized_tfidf:
                candidates[(doc_id, term)] = candidates.get((doc_id, term), 0) + tfidf

    return candidates


def generate_document_tfidf_matrix(query_to_search, index, words, pls, DL):
    total_vocab_size = len(index.term_total)
    candidates_scores = get_candidate_documents_and_scores(query_to_search, index, words, pls, DL)
    unique_candidates = np.unique([doc_id for doc_id, freq in candidates_scores.keys()])
    D = np.zeros((len(unique_candidates), total_vocab_size))
    D = pd.DataFrame(D)

    D.index = unique_candidates
    D.columns = index.term_total.keys()

    for key in candidates_scores:
        tfidf = candidates_scores[key]
        doc_id, term = key
        D.loc[doc_id][term] = tfidf
    return D


def cosine_similarity(D, Q):
    cosSim_dict = {}
    docs = D.to_numpy()
    docid = 0
    for doc in docs:
        mone = np.dot(doc, Q)
        mechane = np.linalg.norm(doc) * np.linalg.norm(Q)
        cosSim = mone / mechane
        cosSim_dict[D.index[docid]] = cosSim
        docid += 1
    return cosSim_dict


def get_top_n(sim_dict):
    lst= sorted([(doc_id, builtins.round(score, 5)) for doc_id, score in sim_dict.items()], key=lambda x: x[1],
                  reverse=True)[:100]
    id_title_dct = InvertedIndex.read_index('title', 'index_title_dct')
    return [(int(x[0]), id_title_dct.get(x[0], 0)) for x in lst]


def get_topN_score_for_queries(clean_query, index):
    words, pls = get_posting_gen(index)
    DL = InvertedIndex.read_index('text', 'index_text_dct')
    Q = generate_query_tfidf_vector(clean_query, index, DL)
    D = generate_document_tfidf_matrix(clean_query, index, words, pls, DL)
    cosSim_dict = cosine_similarity(D, Q)
    topN = get_top_n(cosSim_dict)
    return topN

In [ ]:
# clean_queries=['similarity', 'laws', 'must', 'obeyed', 'constructing', 'aeroelastic', 'models', 'heated', 'high', 'speed', 'aircraft']
# idx_body =InvertedIndex.read_index('text','index_text')
# tfidf_queries_score = get_topN_score_for_queries(clean_queries,idx_body)
# #print(tfidf_queries_score)

**Page Rank**

In [ ]:
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the 
      second entry is the destination page id. No duplicates should be present. 
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph 
      created by the wikipedia links. No duplicates should be present. 
  '''
  source = pages.map(lambda x: (x[0],))
  target = pages.flatMap(lambda x: map(lambda y: (y[0],) ,x[1]))
  vertices = (source.union(target)).distinct()
  edges = (pages.flatMap(lambda x: list(map(lambda y: (x[0],y[0]),x[1])))).distinct()
  return edges, vertices


In [ ]:

# print(doc_anchor_text_before.take(5))

# print(edges.take(5))
# print(vertices.take(5))

**write dictionary to disk**

In [ ]:
# #doc_anchor_text_before = parquetFile.limit(1000).select("anchor_text","id").rdd
# edges, vertices = generate_graph(doc_anchor_text_before)
# edgesDF = edges.toDF(['src', 'dst']).repartition(4, 'src')
# verticesDF = vertices.toDF(['id']).repartition(4, 'id')
# g = GraphFrame(verticesDF, edgesDF)


In [ ]:
# pr_results = g.pageRank(resetProbability=0.15, maxIter=10)
# pr = pr_results.vertices.select("id", "pagerank")
# pr = pr.sort(col('pagerank').desc())
# wid2pr = {}
# for row in pr.rdd.toLocalIterator():
#     wid2pr[row[0]] = row[1]
  

# with open('./pagerank', 'wb') as f:# write out the counter as binary file (pickle it)
#   pickle.dump(wid2pr, f)
# # a=pr.repartition(1).write.csv('test', compression="gzip")
# # pr.show()

In [ ]:

# with open('./pagerank', 'rb') as f: # read in the dictionary from disk
#   wid2pr = pickle.loads(f.read())


In [ ]:
# print(list(wid2pr.keys())[:5])

# #part-00000-c7edd075-ec62-4d10-9694-d2a95242a2ea-c000.csv.gz
# #print(pr.repartition(1).read.csv('part-00000-c7edd075-ec62-4d10-9694-d2a95242a2ea-c000.csv.gz'))
# # df = spark.read.format("csv").load("/content/test/part-00000-c7edd075-ec62-4d10-9694-d2a95242a2ea-c000.csv.gz")
# # df.show()
# # df
# # print(df[2])

In [ ]:
#print(pr.rdd.take(10)) # need to return some 

**search_title() and search_anchor()**

---

In [ ]:
def search_by_binary(clean_query,index):
  words,pls = get_posting_gen(index)
  candidates = []
  for term in np.unique(clean_query):        
    if term in words:  
      candidates = candidates + list(map(lambda x: x[0], pls[words.index(term)]))
  in_order = list(map(lambda x: int(x[0]), (Counter(candidates)).most_common()))
  return in_order

In [ ]:
# #test
# idx_anchor =InvertedIndex.read_index('anchor','index_anchor')
# rel_docs = search_by_binary(["best", "marvel", "movie", "i", "ever", "love", "where", "does", "vanilla", "flavoring" ,"come" ,"from", "best" ,"ice" ,"cream", "flavour"] , idx_anchor)
# print(rel_docs)

**Page views**

In [ ]:
# # Paths
# # Using user page views (as opposed to spiders and automated traffic) for the 
# # month of August 2021
# pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
# p = Path(pv_path) 
# pv_name = p.name
# pv_temp = f'{p.stem}-4dedup.txt'
# pv_clean = f'{p.stem}.pkl'
# # Download the file (2.3GB) 
# !wget -N $pv_path
# # Filter for English pages, and keep just two fields: article ID (3) and monthly 
# # total number of page views (5). Then, remove lines with article id or page 
# # view values that are not a sequence of digits.
# !bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp  #HORRIBLE DOWNLOAD

# wid2pv = Counter() # Create a Counter (dictionary) that sums up the pages views for the same article, resulting in a mapping from article id to total page views.
# with open(pv_temp, 'rt') as f:
#   for line in f:
#     parts = line.split(' ')
#     wid2pv.update({int(parts[0]): int(parts[1])})

# with open('./pageview', 'wb') as f:# write out the counter as binary file (pickle it)
#    pickle.dump(wid2pv, f)

#FRONTEND
# with open('./pageview', 'rb') as f: # read in the dictionary from disk
#   wid22pv = pickle.loads(f.read())

# keys = list(wid22pv.keys())
# for i in range(10):
#   key=keys[i]
#   print((key,wid22pv.get(key,0)))


In [ ]:
# def most_viewed(wikiIDs):
#   lst=[]
#   for id in wikiIDs:
#     lst.append(wid22pv.get(id,0))
#   return (lst)

In [ ]:
# pages=[600744,59804426,4838455,7777] #list of doc id's need to write to disk
# print(pages)
# pages_ranked = most_viewed(pages)
# print(pages_ranked)

Search Function

---

1. New indexes for title and body using stemming and remove stopwords, and?
2. bm25 data preprocess
3. bm25 calc and other calculations?

4. נחשיב רק מסמכים שמכילים מושגים שה-idf שלהם גדול יותר מאיזשהו threashold שנגדיר
or - sort the posting list by tf-idf

In [ ]:
#    rdd:     doc_title_pairs   ,    doc_text_pairs
!mkdir search_text search_title

In [ ]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

def improved_tokenize(text):
    list_of_tokens =  [stemmer.stem(token.group()) for token in RE_WORD.finditer(text.lower()) if token.group() not in all_stopwords] ##is it work?
    return list_of_tokens


def improved_word_count(text, id):
  tokens = [stemmer.stem(token.group()) for token in RE_WORD.finditer(text.lower())]
  countTokens = OrderedDict(Counter(tokens))
  dict_keys = list(countTokens.keys())
  for token in dict_keys:
    if token in all_stopwords:
      countTokens.pop(token)
  return list(map(lambda x: (x , (id,countTokens[x])),countTokens.keys()))

def improved_reduce_word_counts(unsorted_pl):
  return sorted(unsorted_pl, key=lambda x: x[1], reverse=True)[:100]



def createAndWriteImprovedIndex(doc_rdd,directory,name):
  inverted = InvertedIndex()
  word_counts = doc_rdd.flatMap(lambda x: word_count(x[0], x[1]))
  postings = word_counts.groupByKey().mapValues(improved_reduce_word_counts)
  # #postings_filtered = postings.filter(lambda x: len(x[1])>10)
  df_ = calculate_df(postings)
  inverted.df = df_.collectAsMap()
  word_counter = postings.map(lambda x: (x[0], builtins.sum([y[1] for y in x[1]])))
  inverted.term_total = Counter(word_counter.collectAsMap())
  posting_locs_list = partition_postings_and_write(postings,directory).collect()  ##collect? 
  inverted.posting_locs = countForDict(posting_locs_list)
  word_counts_length = doc_rdd.map(lambda x: doc_count(x[0], x[1]))
  inverted.dct = word_counts_length.collectAsMap()
  writeIdx(inverted, directory, name)
  return inverted


In [ ]:
from BM25_from_index import *

def search_back(clean_query):
  idx_title = InvertedIndex.read_index('search_title', 'index_title')
  idx_text = InvertedIndex.read_index('search_text', 'index_text')
  #topNtitle = get_topN_score_for_queries(clean_query,idx_title)
  #topNtext = get_topN_score_for_queries(clean_query,idx_text)
  dct_title = InvertedIndex.read_index('title', 'index_title_dct')
  DL_title =  InvertedIndex.read_index('search_title', 'index_title_dct')
  DL_text = InvertedIndex.read_index('search_text', 'index_text_dct')
  bm25title = BM25_from_index(idx_title,DL_title)
  bm25text = BM25_from_index(idx_text,DL_text)
  title_candidates = bm25title.search(clean_query)
  text_candidates = bm25text.search(clean_query)
  best_text_title = merge_results(title_candidates , text_candidates , N=100)
  return list(map(lambda tup: (int(tup[0]), dct_title[tup[0]]) , best_text_title))
  

def merge_results(title_scores,body_scores,title_weight=0.5,text_weight=0.5,N = 3):    
    # YOUR CODE HERE
    title_dict = dict(title_scores)
    body_dict = dict(body_scores)
    docs_id_for_query = set(list(title_dict.keys()) + list(body_dict.keys()))
    allscores = []
    for docid in docs_id_for_query:
      allscores.append((docid,title_dict.get(docid,0)*title_weight + body_dict.get(docid,0)*text_weight))
    score_sorted = sorted(allscores,key=lambda x:x[1],reverse = True)[:N]
    return score_sorted

In [ ]:
title_index = createAndWriteImprovedIndex(doc_title_pairs,'search_title','index_title')
text_index = createAndWriteImprovedIndex(doc_text_pairs,'search_text','index_text')

In [ ]:
query = 'data science'
tokens = improved_tokenize(query)
print(search_back(tokens))

[(2349, 'Abstract data type'), (2052, 'Array data structure'), (2246, 'Analgesic'), (2380, 'Accelerated Graphics Port'), (1368, 'Assembly language'), (856, 'Apple Inc.'), (634, 'Analysis of variance'), (2316, 'Audio file format'), (1063, 'Algorithms for calculating variance'), (2078, 'Acorn Electron'), (775, 'Algorithm'), (586, 'ASCII'), (1451, 'APL (programming language)'), (1164, 'Artificial intelligence'), (612, 'Arithmetic mean'), (2230, 'Analysis of algorithms'), (1260, 'Advanced Encryption Standard'), (1094, 'Economy of Armenia'), (2141, 'Atari ST'), (1134, 'Analysis'), (2115, 'AppleTalk'), (1967, 'Apollo 12'), (1092, 'Demographics of Armenia'), (2039, 'Avionics'), (2116, 'Apple II series'), (308, 'Aristotle'), (1914, 'Antimicrobial resistance'), (1955, 'Adobe Inc.'), (1969, 'Apollo 15'), (1971, 'Apollo 17'), (569, 'Anthropology'), (2114, 'IBM AIX'), (2142, 'List of artificial intelligence projects'), (663, 'Apollo 8'), (1242, 'Ada (programming language)'), (738, 'Albania'), (127